**When exploring a large set of documents such as Wikipedia, news articles, stackoverflow, etc. it can be useful to find the most similar material.**

**In this task we are going to use K-nearest neighbor algorithm to retrieve the most relevant K documents by the words mentioned in our documnet**

**We will use brute_force KNN to find the exact NN, and we will use both methods:**
1. Naive Word count document representation.
2. TF-IDF document representation.
___

In [1]:
DATA_PATH = "../data/people_wiki.csv"

In [2]:
# Load packages
import pandas as pd
import numpy as np

import re
import string
from nltk.corpus import stopwords

from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors

pd.set_option("display.max_colwidth", 500)

___
## Functions

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words("english"))
TRANSLATOR=str.maketrans({k:'' for k in string.punctuation})

def text_prepare(text):
    """
    text: a string to process
    return: modified initial string
    """
    text = text.lower()
    text = text.translate(TRANSLATOR)
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS])
    return text

In [4]:
def top_words(name):
    """
    Get the most frequent words for the celeberity page
    """
    result = Counter()
    row = df[df['name']==name]['word_counts'].iloc[0]
    for key in row.keys():
        if key in SORT_BY_FREQ:
            result[key] = row[key]

    df_ = pd.DataFrame(result.most_common(10), columns=['word', 'count'])
    return df_

In [5]:
def get_top_n_words(name, vectorizer, count_matrix, n):
    
    index = df[df['name'] == name].index[0]
    feature_array = np.array(vectorizer.get_feature_names_out())
    sorting = np.argsort(count_matrix[index].toarray()).flatten()[::-1]

    top_n = feature_array[sorting][:n]
    counts = count_matrix[index].toarray()[0, sorting[:n]]

    result = pd.DataFrame({"word":top_n, "vlaue": counts})
    return result

___
## Read Data:

In [6]:
raw_df = pd.read_csv(DATA_PATH)

In [7]:
# Dsiplay shape of df
raw_df.shape

(59071, 3)

In [8]:
raw_df.sample(3)

,URI,name,text
25011,<http://dbpedia.org/resource/Hilary_Tann>,Hilary Tann,hilary tann born 1947 is a welsh composer now based in the united statestann holds degrees in music composition from the university of wales cardiff and princeton university her compositions are published by oxford university press tanns orchestral works have been released on the northsouth recordings cd here the cliffs music of great integrity impeccable craft and genuine expressive ambition robert carl fanfare 36i her overture with the heather and small birds commissioned by the 1994 cardi...
33644,<http://dbpedia.org/resource/Justin_Jeffre>,Justin Jeffre,justin paul jeffre born on february 25 1973 is an american pop singer and politician a longtime resident and vocal supporter of cincinnati jeffre is probably best known as a member of the multiplatinum selling boy band 98 degreesbefore shooting to super stardom jeffre was a student at the school for creative and performing arts in cincinnati it was there that he first became friends with nick lachey the two would later team up with drew lachey and jeff timmons to form 98 degrees the group wa...
54620,<http://dbpedia.org/resource/Jeffrey_Jordan>,Jeffrey Jordan,jeffrey michael jordan born november 18 1988 is an american former basketball player who played for the university of central florida knights and the university of illinois fighting illini he played high school basketball for loyola academy in wilmette illinois jordan is the elder son of retired nba mvp michael jordan who played for the chicago bulls and washington wizards and the older brother of marcus jordan jeffrey jordan has been the subject of local and national media attention and had...


___
## Make copy and prepare text:
**We will remove stopwords and punctuations and clean our text to prepare it for constructing BOW**

In [9]:
df = raw_df.copy()

In [10]:
df['clean_text'] = df['text'].apply(text_prepare)

In [11]:
df.sample(5)

,URI,name,text,clean_text
28908,"<http://dbpedia.org/resource/Tommy_Jackson_(footballer,_born_1946)>","Tommy Jackson (footballer, born 1946)",thomas tommy jackson born 3 november 1946 in belfast is a former northern irish footballer who played as a midfielder for everton nottingham forest and manchester united he also amassed a total of 35 caps for the northern ireland national football team following his playing career he went into management taking charge of various clubs in both northern ireland and the republic of irelandjackson began his professional football career playing for glentoran he became a regular in the glentoran s...,thomas tommy jackson born 3 november 1946 belfast former northern irish footballer played midfielder everton nottingham forest manchester united also amassed total 35 caps northern ireland national football team following playing career went management taking charge various clubs northern ireland republic irelandjackson began professional football career playing glentoran became regular glentoran side age 21 leaguewinners medals 1967 1968 signing everton february 1968 two seasons toffees jac...
13022,<http://dbpedia.org/resource/Kristine_W>,Kristine W,kristine weitz born june 8 1962 known by the stage name of kristine w is an american singersongwriter she has released seven albums in 2004 the advocate stated that she had helped shape the nightlife of the past decade her first 8 singles all reached 1 on the billboard hot dance club play charts which set a new record as of 2009 14 of 15 singles had reached the top of the billboard dance chartsweitz was born and raised in pasco washington her mother was a performer weitz won the miss washing...,kristine weitz born june 8 1962 known stage name kristine w american singersongwriter released seven albums 2004 advocate stated helped shape nightlife past decade first 8 singles reached 1 billboard hot dance club play charts set new record 2009 14 15 singles reached top billboard dance chartsweitz born raised pasco washington mother performer weitz miss washington 19801981 title swimsuit talent miss america pageant moved las vegas enrolled university nevada las vegas graduated ba communica...
24529,<http://dbpedia.org/resource/Shamsher_Singh_Sandhu>,Shamsher Singh Sandhu,shamsher singh sandhu ma medborn march 31937 is a well known canadian of punjabi origin gazal writer or gazalgo he is best known for his beautiful gazals he lives in calgary alberta canada since 1997 he learned the art of writing gazal and started writing gazals in 2002 after the age of 65 since then he has published 6 books containing 513 gazals as follows 1 ga zindgi de geet toon printed published in calgary 2003 2 jot sahas de jaga printed published in calgary 2005 3 ban shua toon printed...,shamsher singh sandhu medborn march 31937 well known canadian punjabi origin gazal writer gazalgo best known beautiful gazals lives calgary alberta canada since 1997 learned art writing gazal started writing gazals 2002 age 65 since published 6 books containing 513 gazals follows 1 ga zindgi de geet toon printed published calgary 2003 2 jot sahas de jaga printed published calgary 2005 3 ban shua toon printed published calgary 2006 4 roshni de bhal printed published calgary 2007 5 sulagdi lee...
52874,<http://dbpedia.org/resource/Stephan_P._Mickle>,Stephan P. Mickle,stephan p mickle born 1944 is an american lawyer and judge of the united states district court for the northern district of florida mickle was born in new york city in 1965 he received his bachelor of arts degree in political science he was the first black student to graduate from the university of florida in 1966 he received his med from the university of florida additionally he received his jd from the university of florida college of law in 1970 he was the second black student to graduate...,stephan p mickle born 1944 american lawyer judge united states district court northern district florida mickle born new york city 1965 received bach

**Now construct a dictionary of the most frequent words after preparing text**

In [12]:
words_count = Counter()

for text in df['clean_text']:
    for word in text.split():
        words_count[word] += 1

In [33]:
DICT_SIZE = 7000

# find the most common words in all the corpus
SORT_BY_FREQ = [x[0] for x in words_count.most_common(DICT_SIZE)]

**The above dictionary has the top 50000 frequent words in the whole corpus and so we can use it in our vectorizer**

### Now construct the BOW for the text

In [34]:
# We will use the most common words to use less space and be more accurate
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b', vocabulary=SORT_BY_FREQ)

word_count_matrix = vectorizer.fit_transform(df['clean_text'])

### Now construct our KNN model 

In [35]:
model = NearestNeighbors(metric="euclidean", algorithm="brute")

In [36]:
model.fit(word_count_matrix)

NearestNeighbors(algorithm='brute', metric='euclidean')

**Now find the most relevant wiki page to the person we search for**
for example we are searching for the people who are related to 'Barack Obama' and so we will find the nearest 10 characters whose wiki page are the most similar to Obama's page and so on

first we will find the index of Obama's page and then we will use it in our model and so it will search in the entire corpus and so it will find the top 10 characters and retrieve it, then we will construct a dataframe and join it with the original dataframe using indices to retrieve the most common characters

In [37]:
# We will wright the name of the celebrity we are looking for and get its index to use it with our BOW
index = df[df['name'] == "Barack Obama"].index[0]
index

35817

**Now use the model to find the most relevant neighbors**

In [38]:
distances, indices = model.kneighbors(word_count_matrix[index], n_neighbors=10)

In [39]:
neighbors = pd.DataFrame({"id":indices[0].tolist(), 
                          "distance":distances[0].tolist()})
neighbors.set_index("id", inplace=True)


# Join both the dataframes and so it will show us the best results
neighbors.join(df)[['name']+["distance"]]

,name,distance
id,,
35817,Barack Obama,0.000000
24478,Joe Biden,23.237900
50452,John F. Tierney,24.899799
11517,Louis Susman,25.039968
57635,Joe Sestak,25.357445
53303,Juan F. Vasquez,25.396850
43713,Ken Salazar,25.455844
33417,Tulsi Gabbard,25.553865
16880,Cynthia Hogan,25.651511


#### Now analyse the most frequent words in each document to see why those celebrities are too close

In [40]:
get_top_n_words("Barack Obama", vectorizer, word_count_matrix, 10)

,word,vlaue
0,obama,9
1,act,8
2,us,6
3,law,6
4,president,4
5,military,4
6,control,4
7,iraq,4
8,democratic,4
9,signed,3


In [41]:
get_top_n_words("Joe Biden", vectorizer, word_count_matrix, 10)

,word,vlaue
0,act,5
1,us,5
2,president,5
3,vice,5
4,elected,4
5,obama,4
6,senator,3
7,committee,3
8,united,3
9,war,3


In [42]:
get_top_n_words("Jeff Sessions", vectorizer, word_count_matrix, 10)

,word,vlaue
0,republican,4
1,alabama,4
2,elected,3
3,states,3
4,sessions,3
5,district,3
6,us,3
7,senators,2
8,southern,2
9,united,2


___
## Now trying the tfidf transformation to get more relevant and accurate results

In [23]:
# Gonfigure the vectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9,
                                   ngram_range=(1, 2),
                                   token_pattern='(\S+)')

In [24]:
# Construct the tfidf tokens matrix
tfidf_counts = tfidf_vectorizer.fit_transform(df['clean_text'])

**Now define the model**

In [25]:
model = NearestNeighbors(metric="euclidean", algorithm="brute")

In [26]:
model.fit(tfidf_counts)

NearestNeighbors(algorithm='brute', metric='euclidean')

In [27]:
# We will wright the name of the celebrity we are looking for and get its index to use it with our BOW
index = df[df['name'] == "Barack Obama"].index[0]
index

35817

**Now use the model to find the most relevant neighbors**

In [28]:
distances, indices = model.kneighbors(tfidf_counts[index], n_neighbors=10)

In [29]:
neighbors = pd.DataFrame({"id":indices[0].tolist(), 
                          "distance":distances[0].tolist()})
neighbors.set_index("id", inplace=True)


# Join both the dataframes and so it will show us the best results
neighbors.join(df)[['name']+["distance"]]

,name,distance
id,,
35817,Barack Obama,0.000000
24478,Joe Biden,1.219453
57108,Hillary Rodham Clinton,1.284820
2412,Joe the Plumber,1.287304
46811,Jeff Sessions,1.287454
18827,Henry Waxman,1.288715
38714,Eric Stern (politician),1.292733
48693,Artur Davis,1.293186
38376,Samantha Power,1.293938


#### Now analyse the most frequent words in each document to see why those celebrities are too close

In [30]:
get_top_n_words("Barack Obama", tfidf_vectorizer, tfidf_counts, 10)

,word,vlaue
0,obama,0.306565
1,act,0.209201
2,us military,0.127864
3,iraq,0.127498
4,law,0.121518
5,control,0.110742
6,act 2010,0.110500
7,us,0.103164
8,ordered,0.102782
9,military,0.100362


In [31]:
get_top_n_words("Joe Biden", tfidf_vectorizer, tfidf_counts, 10)

,word,vlaue
0,biden,0.477079
1,obama,0.154951
2,act,0.148696
3,vice,0.135767
4,vice president,0.119130
5,resolved,0.100779
6,judiciary committee,0.100621
7,senator,0.098132
8,us,0.097769
9,president,0.093588


In [32]:
get_top_n_words("Hillary Rodham Clinton", tfidf_vectorizer, tfidf_counts, 10)

,word,vlaue
0,clinton,0.371161
1,first lady,0.282296
2,lady,0.206021
3,secretary state,0.166982
4,rodham,0.144931
5,us,0.114570
6,secretary,0.111522
7,lady united,0.110607
8,state,0.105855
9,first,0.104117


___
# IN THIS NOTEBOOK:
**We figured out that we can use KNN as an algorithm for information retrival, but we found that it takes a long time to get the K-neighbors and with millions of records; this brute force algorithm would be trivial**

`concluding that we need a more sofisticated algorithm with as fast speed as possible, so we wont use exact NN approach but instead the approximated KNN approach; either with KD-tree or using the more robust LSH algorithm (Locality Sensitive Hashing).`